# BB84 Basic
## Introduction
BB84 is a Quantum Key Distribution (QKD) protocol. It is a method of securely distributing cryptographic keys over a distance using quantum mechanics. It is named after its inventors Charles Bennett and Gilles Brassard in 1984.

## Works on
It works on quantum mechanics protocol such as No Clonning, Superposition, and Unceratinity Principle.

## Alice and Bob
Suppose there are 3 characters Alice, Bob and Eve. Alice and Bob are trying to communicate successfully without any interuption in their private channel. For that they transfer their Basis choice on the public channel.
After Bob receives the basis he can compare it with the qubits he received and if the basis matches then he can measure the qubit and get the correct bit. If the basis doesn't match then he will get a random bit. After that Alice and Bob can compare their bits on the public channel and if they match then they can use those bits as their key. If they don't match then they can discard those bits and try again.

## Eve the Evasdropper
If Eve is listening to the public channel then she can measure the basis and qubits which are being sent by the Alice and can understand the key. What will happen is if Eve is evasdropping than the bits which are being evasdropped by Eve might match or might not match. She cannot be sure about the bits she received but she has to forward the bits which are in some way so that Bob can reveive them. And if she forwards the bits which are not correct then Alice and Bob will come to know that someone is listening to their channel.

In [148]:
from random import choices
import cirq

## Declaring Qubits and Circuit
First, we declare the qubits and the circuit. We will use qubits for the BB84 protocol.

In [149]:
# Declaring qubits and Circuit
num_qubits = 5 
qubits = cirq.NamedQubit.range(num_qubits, prefix='q')
alice_circuit = cirq.Circuit()
bob_circuit = cirq.Circuit()

## Declaring Bases and Keys
**Bases** : Bases are like special glasses that help you see hidden things. But here, instead of glasses, they're directions, like up and sideways. You and your friend agree on these directions, but you keep them secret from others.

These are orientations used to measure the state of a quantum bit (qubit). In BB84, the two commonly used bases are usually represented as the rectilinear basis (horizontal and vertical) and the diagonal basis (45 degrees and 135 degrees).

Bases are used to measure the state of a qubit. If the basis used to measure a qubit is the same as the basis used to prepare the qubit, then the measurement will be correct. If the bases do not match, then the measurement will be random (will not match).

**Keys** : Keys are like the secret codes you and your friend make using these directions. You have a bunch of secret codes, and your friend has the same ones, but nobody else knows what they are because they don't have the special glasses (bases) you used to create them. These codes help you send secret messages that only you and your friend can understand.

They are the secret strings of bits generated and shared between two parties using quantum bits. These bits form the basis of secure communication.

Keys are used to verify that have both parties have received the same information. They are also used to encrypt and decrypt messages.

In [150]:
encoded_gates = {0: cirq.I, 1: cirq.X} # this specify which gates to apply based on the bit value
basis_gates = {'Z': cirq.I, 'X': cirq.H} # this specify which gates to apply based on the basis
print(encoded_gates)
print(basis_gates)

{0: cirq.I, 1: cirq.X}
{'Z': cirq.I, 'X': cirq.H}


In [151]:
## Declaring Bases and Gates
alice_key = choices([0,1], k = num_qubits) # Alices Key / Alice randomly chooses bits
alice_base = choices(['Z', 'X'], k = num_qubits) # Alices Base / Alice randomly chooses bases

print(alice_key)
print(alice_base)

[0, 0, 1, 1, 0]
['X', 'X', 'X', 'Z', 'Z']


## Alice creates Qubits
Alice will create qubits based on the Bases and Gates she has choosen randomly.
Here for every Gates and Bases she has an encoded_gate and encoded_basis.

In [152]:
# Alice creating Qubits
for bit in range(num_qubits):
    alice_encode_value = alice_key[bit] # to get the value of alice_key, whether its 0 or 1.
    alice_encode_gate = encoded_gates[alice_encode_value] # to decide value of 0 and 1 for e.g : 0 = cirq.I and 1 = cirq.X
    #print(alice_encode_gate)
    alice_basis_value = alice_base[bit] # to get the value of alice_base, whether its Z or X.
    alice_bases_gates = basis_gates[alice_basis_value] # to decide value of Z and X for e.g : Z = cirq.I and X = cirq.H
    #print(alice_basis_value)

    alice_qubit = qubits[bit] # for every qubits[] in qubits i.e q0, q1, q2, q3, q4.
    alice_circuit.append(alice_encode_gate(alice_qubit)) # Append alice_encode_gate on alice_circut for every alice_qubit
    alice_circuit.append(alice_bases_gates(alice_qubit)) # Append alice_bases_gate on alice_circut for every alice_qubit

print(alice_circuit)

q0: ───I───H───

q1: ───I───H───

q2: ───X───H───

q3: ───X───I───

q4: ───I───I───


# Alice Sends Qubits to Bob
Alice sends the qubits to Bob through the public channel. No coding required for this step. (Assume that we have sent the qubits to Bob)

In [153]:
# Bob picks bases
bob_bases = choices(['Z', 'X'], k = num_qubits)

for bit in range(num_qubits):
    bob_basis_value = bob_bases[bit]
    bob_basis_gate = basis_gates[bob_basis_value]

    bob_qubit = qubits[bit]
    bob_circuit.append(bob_basis_gate(bob_qubit))

print(bob_circuit)

q0: ───I───

q1: ───H───

q2: ───I───

q3: ───I───

q4: ───H───


In [154]:
# Bob measures qubits
bob_measure = bob_circuit.append(cirq.measure(qubits, key= 'bob key')) # measures the bob_circuit
print(bob_measure)

None


In [155]:
# Bob Creates his key
bb84_circuit = alice_circuit + bob_circuit # don't know why we do this

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0] # measures the q0 which has measurement qubit and prints the bob_key
print(bob_key)

[1 0 0 1 1]


## Alice and Bob Compare their Bases
Alice and Bob compare their bases on the public channel. If the bases match then they will keep the qubits and if the bases don't match then they will discard the qubits.

In [156]:
# Alice and Bob Compares
final_alice_key = []
final_bob_key = []

for bit in range(num_qubits):
    if alice_base[bit] == bob_bases[bit]:
        final_alice_key.append(alice_key[bit]) # will only add which matches with bob's key
        final_bob_key.append(bob_key[bit]) # will only add which matches with alice's key

print(final_alice_key)
print(final_bob_key)

[0, 1]
[0, 1]


In [157]:
# Checking for Eve as evasdropper
if final_alice_key[0] == final_bob_key[0]:
    final_alice_key = final_alice_key[1:]
    final_bob_key = final_bob_key[1:]
    print("We can use our keys!")
else:
    print("Eve has been evasdropping!")

We can use our keys!
